<a href="https://colab.research.google.com/github/DmitryKutsev/ml_hw/blob/master/hw3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget http://qwone.com/~jason/20Newsgroups/20news-19997.tar.gz

--2020-02-28 11:20:26--  http://qwone.com/~jason/20Newsgroups/20news-19997.tar.gz
Resolving qwone.com (qwone.com)... 108.20.201.166
Connecting to qwone.com (qwone.com)|108.20.201.166|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 17332201 (17M) [application/x-gzip]
Saving to: ‘20news-19997.tar.gz’

20news-19997.tar.gz 100%[===================>]  16.53M  6.66MB/s    in 2.5s    

2020-02-28 11:20:28 (6.66 MB/s) - ‘20news-19997.tar.gz’ saved [17332201/17332201]



In [2]:
!gunzip 20news-19997.tar.gz

gzip: 20news-19997.tar already exists; do you wish to overwrite (y or n)? т
	not overwritten


In [3]:
!ls

20news-19997.tar  20news-19997.tar.gz  sample_data


In [0]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.datasets import fetch_20newsgroups
count_vect = CountVectorizer()
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold, cross_val_score, GridSearchCV, RepeatedStratifiedKFold
import pandas as pd
import matplotlib.pyplot as plt

In [0]:
categories = ['alt.atheism', 'soc.religion.christian', 'comp.graphics', 'sci.med']
twenty_train = fetch_20newsgroups(subset='train', categories=categories, shuffle=True, random_state=42)

In [8]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(twenty_train.data) #Learn the vocabulary dictionary and return term-document matrix.
X_train_counts.shape

(2257, 35788)

In [0]:
from sklearn.neighbors import KNeighborsClassifier
kneighbors = KNeighborsClassifier().fit(X_train_counts, twenty_train.target)

In [12]:
from sklearn.metrics import *
twenty_test = fetch_20newsgroups(subset='test', categories=categories, 
                                 shuffle=True, random_state=0)
X_test = count_vect.transform(twenty_test.data) # Extract token counts out of raw text documents using the vocabulary fitted with fit 
print(X_test.shape)
predicted = kneighbors .predict(X_test)
predicted_proba = kneighbors.predict_proba(X_test)
predicted_proba[:5]

(1502, 35788)


array([[0. , 0.6, 0. , 0.4],
       [0.2, 0. , 0. , 0.8],
       [0. , 0. , 0. , 1. ],
       [0.2, 0. , 0. , 0.8],
       [0. , 0.4, 0.4, 0.2]])

In [0]:
from sklearn.model_selection import train_test_split, KFold

In [0]:
n_fold = 10
folds = KFold(n_splits=n_fold, shuffle=True, random_state=0)
stratified_folds = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=0)
repeated_folds = RepeatedStratifiedKFold(n_splits=n_fold, n_repeats=20, random_state=0)

In [48]:
parameter_grid = {'weights' : ['uniform', 'distance'],
                  'algorithm' : ['auto', 'ball_tree', 'kd_tree', 'brute'],
                  'leaf_size' : [20, 15, 25],
                  'n_neighbors': [5,4,6]
                 }

grid_search = GridSearchCV(kneighbors, param_grid=parameter_grid, cv=folds)
grid_search.fit(X_train_counts, twenty_train.target)
print('Best score: {}'.format(grid_search.best_score_))
print('Best parameters: {}'.format(grid_search.best_params_))

/usr/local/lib/python3.6/dist-packages/sklearn/neighbors/_base.py:413: UserWarning: cannot use tree with sparse input: using brute force
  warnings.warn("cannot use tree with sparse input: "
/usr/local/lib/python3.6/dist-packages/sklearn/neighbors/_base.py:413: UserWarning: cannot use tree with sparse input: using brute force
  warnings.warn("cannot use tree with sparse input: "
/usr/local/lib/python3.6/dist-packages/sklearn/neighbors/_base.py:413: UserWarning: cannot use tree with sparse input: using brute force
  warnings.warn("cannot use tree with sparse input: "
/usr/local/lib/python3.6/dist-packages/sklearn/neighbors/_base.py:413: UserWarning: cannot use tree with sparse input: using brute force
  warnings.warn("cannot use tree with sparse input: "
/usr/local/lib/python3.6/dist-packages/sklearn/neighbors/_base.py:413: UserWarning: cannot use tree with sparse input: using brute force
  warnings.warn("cannot use tree with sparse input: "
/usr/local/lib/python3.6/dist-packages/sklear

Best score: 0.7275339233038347
Best parameters: {'algorithm': 'auto', 'leaf_size': 20, 'n_neighbors': 4, 'weights': 'distance'}


In [0]:
from sklearn import linear_model

In [55]:
lr = linear_model.LogisticRegression().fit(X_train_counts, twenty_train.target)

parameter_grid = {'class_weight' : ['balanced', None],
                  'penalty' : ['l2', 'l1'],
                  'solver' : ['liblinear', 'saga'],
                  'C' : [0.001, 0.01, 0.08],
                  'max_iter': [2,10]
                 }

grid_search = GridSearchCV(lr, param_grid=parameter_grid, cv=folds)
grid_search.fit(X_train_counts, twenty_train.target)
print('Best score: {}'.format(grid_search.best_score_))
print('Best parameters: {}'.format(grid_search.best_params_))

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_sa

Best score: 0.9570285152409046
Best parameters: {'C': 0.08, 'class_weight': 'balanced', 'max_iter': 10, 'penalty': 'l2', 'solver': 'liblinear'}


/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [0]:
from sklearn.tree import DecisionTreeClassifier
tree = DecisionTreeClassifier().fit(X_train_counts, twenty_train.target)
predicted = clf.predict(X_test)

In [57]:


parameter_grid = {'criterion' : ['gini', 'entropy'],
                  'splitter' : ['best', 'random'],
                  'min_samples_split' : [1, 2, 3],
                  'max_features' : ['auto', 'sqrt', 'log2'],
                  'ccp_alpha': [1.0, 2.0, 0.0]
                 }

grid_search = GridSearchCV(tree, param_grid=parameter_grid, cv=folds)
grid_search.fit(X_train_counts, twenty_train.target)
print('Best score: {}'.format(grid_search.best_score_))
print('Best parameters: {}'.format(grid_search.best_params_))

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: min_samples_split must be an integer greater than 1 or a float in (0.0, 1.0]; got the integer 1

  FitFailedWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: min_samples_split must be an integer greater than 1 or a float in (0.0, 1.0]; got the integer 1

  FitFailedWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: min_samples_split must be an integer greater than 1 or a float in (0.0, 1.0]; got the integ

Best score: 0.6694316617502458
Best parameters: {'ccp_alpha': 0.0, 'criterion': 'gini', 'max_features': 'sqrt', 'min_samples_split': 3, 'splitter': 'best'}
